
# Monitoring Data Quality with Great Expectations

This notebook demonstrates how to monitor data quality using the **Great Expectations** library. 
The example uses a simple dataset to validate specific conditions, ensuring that the data meets the required quality standards.


In [382]:

# Install Great Expectations (if not already installed)
#!pip install great_expectations == 0.13.0



## 1. Preamble: Impor dan Inisialisasi

Mengimpor library yang diperlukan dan menginisialisasi context Great Expectations.


In [383]:
import great_expectations as ge
import pandas as pd
from datetime import datetime
from great_expectations.core import ExpectationConfiguration

# Inisialisasi context Great Expectations
context = ge.data_context.DataContext()

# Path ke dataset dan konfigurasi awal
dataset_path = r'D:\gx0.13\updated_trial_Dataset1.csv'
file_path = r'D:\gx0.13\cities.txt'  # Path untuk file lokasi valid

# Membuat suite ekspektasi
suite = context.create_expectation_suite('mixed_check', overwrite_existing=True)


## 2. Data Loading: Membaca Dataset
Memuat dataset dari file CSV menggunakan Pandas dan konfigurasi batch untuk Great Expectations juga ditambahkan di sini untuk mempersiapkan data yang akan divalidasi.

In [384]:
# Memuat dataset menggunakan Pandas
batch = pd.read_csv(dataset_path, index_col=0)

# Menambahkan konfigurasi batch untuk Great Expectations
batch_kwargs = {
    'path': dataset_path,
    'datasource': 'registrasi_data',
    'data_asset_name': 'registration',
    'reader_method': 'read_csv',
    'reader_options': {'index_col': 0}
}
batch = context.get_batch(batch_kwargs, suite)
batch = batch.reset_index()

#Menampilkan dataset: input-an validation checking
batch

,customer_connection_id,name,phone_number,email,customer_status,location,equipment_type,equipment_status,capacity,registration_timestamp,received_timestamp
0,CC0001,Danuja Wibowo,+62 (52) 202 9748,haryantishania@example.com,active,NaN,Fiber Modem,faulty,0,1/2/2025 13:00:00 AM,1/2/2025 1:06
1,CC0002,Zalindra Maulana,+62 (0466) 541-1885,dadiwahyudin@example.org,pending,Kabupaten Sumbawa Barat,4G Router,unknown,1555,1/2/2025 0:50,1/2/2025 0:58
2,CC0003,Emas Nasyidah,NaN,ikinharyanti@example.net,pending,Kabupaten Banjar,Fiber Modem,unknown,1421,1/2/2025 0:40,1/2/2025 0:49
3,CC0004,Azalea Yulianti,-5681,safitrikarta@example.org,pending,Kabupaten Bulukumba,Unknown Device,maintenance,4491,1/2/2025 0:30,1/2/2025 0:40
4,CC0005,"Dalimin Prasasta, M.Farm",+62 (0173) 072-6575,gamanimanullang@example.com,unknown,Kabupaten Ketapang,4G Router,faulty,5362,1/2/2025 0:20,1/2/2025 0:21
5,CC0006,"dr. Cinthia Mansur, S.IP",(083) 239-6369,orajata@example.net,pending,NaN,5G Antenna,maintenance,1668,1/2/2025 0:10,1/2/2025 0:17
6,CC0007,Titin Tarihoran,-1906,maidapratama@example.com,pending,Kabupaten S I A K,Fiber Modem,maintenance,1441,1/2/2025 0:00,1/2/2025 0:09
7,CC0008,Siska Pudjiastuti,(0526) 650 7811,NaN,unknown,Kabupaten Katingan,ABC,faulty,0,1/1/2025 23:50,1/1/2025 23:59
8,CC0009,Vanya Saputra,+62 (44) 896-5619,empluk45@example.org,unknown,NaN,5G Antenna,unknown,704,1/1/2025 23:40,1/1/2025 23:48
9,CC0010,Eka Wulandari,+62 (068) 017 0373,usinaga@example.org,pending,Kabupaten Sorong Selatan,Unknown Device,faulty,1538,1/1/2025 23:30,1/1/2025 23:33


## 3. Validasi Data
Mencakup berbagai jenis validasi untuk memastikan kualitas data berdasarkan enam aspek utama yaitu Completeness (Kelengkapan), Consistency (Konsistensi), Accuracy (Akurasi), Timeliness (Ketepatan Waktu), Validity (Keabsahan), dan Uniqueness (Keunikan). Setiap jenis validasi bertujuan untuk memeriksa kondisi tertentu dalam data, memastikan bahwa data memenuhi standar dan aturan yang telah ditetapkan.

### A. **Completeness (Kelengkapan Data)**
Memastikan semua data yang seharusnya diisi tidak kosong

In [385]:
# A1. KOLOM WAJIB TERISI

# Menampilkan kolom dataset
columns = batch.columns.tolist()
print("Kolom pada dataset:", columns)

# Validasi eksistensi kolom dan nilai null untuk setiap kolom
def validate_columns(batch, columns):
    for column in columns:
        batch.expect_column_values_to_not_be_null(
            column,
            meta={"KPI": "Completeness"}
        )

validate_columns(batch, columns)

Kolom pada dataset: ['customer_connection_id', 'name', 'phone_number', 'email', 'customer_status', 'location', 'equipment_type', 'equipment_status', 'capacity', 'registration_timestamp', 'received_timestamp']


### B. **Consistency (Konsistensi Data)**
Memastikan format dan struktur data seragam.

In [386]:
# B1. FORMAT KOLOM PHONE NUMBER

phone_number_regex = r'^\+62\s?\(?\d{2,4}\)?\s?\d{3,4}[\s-]?\d{3,4}$'

#Gunakan expect_column_values_to_match_regex untuk validasi format nomor telepon
batch.expect_column_values_to_match_regex(
    column="phone_number",
    regex=phone_number_regex,
    meta={"KPI": "Consistency"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Consistency"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 2,
    "missing_percent": 10.0,
    "unexpected_count": 6,
    "unexpected_percent": 30.0,
    "unexpected_percent_nonmissing": 33.33333333333333,
    "partial_unexpected_list": [
      "-5681",
      "(083) 239-6369",
      "-1906",
      "(0526) 650 7811",
      "081vphrffk",
      "080 641 3373"
    ]
  }
}

In [387]:
# B2. FORMAT KOLOM EMAIL

email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'

#Gunakan expect_column_values_to_match_regex untuk validasi format email
batch.expect_column_values_to_match_regex(
    column="email",
    regex=email_regex,
    meta={"KPI": "Consistency"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Consistency"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 1,
    "missing_percent": 5.0,
    "unexpected_count": 1,
    "unexpected_percent": 5.0,
    "unexpected_percent_nonmissing": 5.263157894736842,
    "partial_unexpected_list": [
      "agustinaadityaexample.net"
    ]
  }
}

In [388]:
# B3. KONSTANTA ANTARKOLOM: Jika equipment_status adalah faulty, maka capacity tidak boleh lebih dari 0.

# Tambahkan kolom baru untuk aturan logika
batch["equipment_status_capacity_check"] = (
    (batch["equipment_status"] != "faulty") | (batch["capacity"] == 0)
)

# Validasi aturan logika
batch.expect_column_values_to_be_in_set(
    column="equipment_status_capacity_check",
    value_set=[True],  # Semua nilai dalam kolom ini harus True
    mostly=1.0, # Pastikan aturan berlaku untuk 100% data
    meta={"KPI": "Consistency"} 
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Consistency"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 4,
    "unexpected_percent": 20.0,
    "unexpected_percent_nonmissing": 20.0,
    "partial_unexpected_list": [
      false,
      false,
      false,
      false
    ]
  }
}

In [389]:
# B4. FORMAT WAKTU KONSISTEN: kolom registration_timestamp
timestamp_regex = r"^(0?[1-9]|[12][0-9]|3[01])/(0?[1-9]|1[0-2])/\d{4} ([01]?[0-9]|2[0-3]):[0-5][0-9]$"

# Validasi untuk kolom registration_timestamp
batch.expect_column_values_to_match_regex(
    column="registration_timestamp",
    regex=timestamp_regex,
    meta={"KPI": "Consistency"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Consistency"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1,
    "unexpected_percent": 5.0,
    "unexpected_percent_nonmissing": 5.0,
    "partial_unexpected_list": [
      "1/2/2025  13:00:00 AM"
    ]
  }
}

In [390]:
# B5. FORMAT WAKTU KONSISTEN: kolom received_timestamp

# Validasi untuk kolom received_timestamp
batch.expect_column_values_to_match_regex(
    column="received_timestamp",
    regex=timestamp_regex,
    meta={"KPI": "Consistency"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Consistency"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1,
    "unexpected_percent": 5.0,
    "unexpected_percent_nonmissing": 5.0,
    "partial_unexpected_list": [
      "1/1/2025  14:23:00 PM"
    ]
  }
}

### C. **Accuracy (Akurasi Data)**
Memastikan data benar secara realitas.

In [391]:
# C1. KISARAN NILAI NUMERIK: capacity harus dalam rentang masuk akal, misalnya 0 hingga 5000.

batch.expect_column_values_to_be_between(
    column='capacity',
    min_value=0,
    max_value=5000,
    meta={"KPI": "Accuracy"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Accuracy"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1,
    "unexpected_percent": 5.0,
    "unexpected_percent_nonmissing": 5.0,
    "partial_unexpected_list": [
      5362
    ]
  }
}

In [392]:
# C2. LOKASI VALID: location harus ada dalam daftar wilayah Indonesia yang valid.

# Membaca daftar kota yang valid dari file
valid_locations = []
file_path = r"D:\gx0.13\cities.txt"  # Sesuaikan path dengan file Anda

# Membaca file txt dan mengisi daftar lokasi
with open(file_path, 'r', encoding='utf-8') as file:
    valid_locations = [line.strip() for line in file.readlines()]

# Menambahkan ekspektasi untuk kolom 'location' agar sesuai dengan lokasi yang valid
batch.expect_column_values_to_be_in_set(
    column='location', 
    value_set=valid_locations,
    meta={"KPI": "Accuracy"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Accuracy"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 7,
    "missing_percent": 35.0,
    "unexpected_count": 1,
    "unexpected_percent": 5.0,
    "unexpected_percent_nonmissing": 7.6923076923076925,
    "partial_unexpected_list": [
      "Kabupaten Jeruk Monyet"
    ]
  }
}

In [393]:
# C3. VALIDASI TIMESTAMP: registration_timestamp tidak boleh di masa depan
#################################################### TIDAK BISA MASUK EXPECTATION ####################################################################

# Fungsi untuk memeriksa dan mengeliminasi timestamp yang tidak valid
def is_valid_timestamp(timestamp_str):
    try:
        
        timestamp = pd.to_datetime(timestamp_str, errors='raise', dayfirst=False)
        
        # Jika timestamp valid, periksa apakah menggunakan AM/PM atau format 24 jam yang tidak sesuai
        if 'AM' in timestamp_str or 'PM' in timestamp_str:
            # Periksa apakah jam lebih dari 12 untuk AM/PM
            hour = timestamp.hour
            if hour > 12:
                return False
        elif timestamp.hour > 23:
            return False
        
        return True
    except Exception:
        return False

# Terapkan fungsi validasi ke kolom 'registration_timestamp' dan filterisasi yang tidak valid
batch["is_valid_registration_timestamp"] = batch["registration_timestamp"].apply(is_valid_timestamp)
valid_registration_batch = batch[batch["is_valid_registration_timestamp"]]

# Validasi Timestamp 'registration_timestamp': pastikan tidak lebih besar dari waktu sekarang
current_time = datetime.now()
valid_registration_batch["registration_timestamp_valid"] = pd.to_datetime(valid_registration_batch["registration_timestamp"], errors='coerce') <= current_time
invalid_registration_rows = valid_registration_batch[valid_registration_batch["registration_timestamp_valid"] == False]

# Validasi dengan Great Expectations untuk 'registration_timestamp'
valid_registration_batch.expect_column_values_to_be_in_set(
    column="registration_timestamp_valid",
    value_set=[True], # Hanya menerima nilai True untuk validasi 'registration_timestamp'
    meta={"KPI": "Accuracy"}
)


{
  "exception_info": null,
  "meta": {
    "KPI": "Accuracy"
  },
  "success": false,
  "result": {
    "element_count": 19,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1,
    "unexpected_percent": 5.263157894736842,
    "unexpected_percent_nonmissing": 5.263157894736842,
    "partial_unexpected_list": [
      false
    ]
  }
}

### D. **Timeliness (Ketepatan Waktu)**
Memastikan data diterima dan dihasilkan tepat waktu.

In [394]:
# D1. SELISIH WAKTU LOGIS: received_timestamp >= registration_timestamp

# Tambahkan kolom validasi untuk perbedaan waktu
batch["timestamp_difference_valid"] = (
    batch["received_timestamp"] >= batch["registration_timestamp"]
)

# Tambahkan expectation untuk memvalidasi kolom ini
batch.expect_column_values_to_be_in_set(
    column="timestamp_difference_valid",
    value_set=[True], # Nilai yang valid adalah True
    result_format="COMPLETE",
    meta={"KPI": "Timeliness"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Timeliness"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 2,
    "unexpected_percent": 10.0,
    "unexpected_percent_nonmissing": 10.0,
    "partial_unexpected_list": [
      false,
      false
    ],
    "partial_unexpected_index_list": [
      10,
      12
    ],
    "partial_unexpected_counts": [
      {
        "value": false,
        "count": 2
      }
    ],
    "unexpected_list": [
      false,
      false
    ],
    "unexpected_index_list": [
      10,
      12
    ]
  }
}

In [395]:
# D2. KETEPATAN WAKTU: Data diterima max. 5 menit after pendaftaran
####################################################### TIDAK BISA MASUK EXPECTATION ###################################################################

# Filter baris dengan timestamp yang valid
valid_rows = batch[
    batch["received_timestamp"].str.match(timestamp_regex, na=False)
    & batch["registration_timestamp"].str.match(timestamp_regex, na=False)
]

# Menghitung Selisih Waktu dalam Menit
valid_rows['time_difference_minutes'] = (
    pd.to_datetime(valid_rows['received_timestamp']) -
    pd.to_datetime(valid_rows['registration_timestamp'])
).dt.total_seconds() / 60

# Menambahkan SLA Kondisi Validasi
SLA_minutes = 5
valid_rows["sla_condition"] = valid_rows["time_difference_minutes"] <= SLA_minutes

# Validasi Kolom SLA
valid_rows.expect_column_values_to_be_in_set(
    column="sla_condition",
    value_set=[True],
    meta={"KPI": "Timeliness"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Timeliness"
  },
  "success": false,
  "result": {
    "element_count": 18,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 9,
    "unexpected_percent": 50.0,
    "unexpected_percent_nonmissing": 50.0,
    "partial_unexpected_list": [
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false,
      false
    ]
  }
}

### E. **Validation (Keabsahan Data)**
Memastikan data sesuai dengan aturan atau domain tertentu.

In [396]:
# E1. KATEGORI VALID: customer_status must have valid value

# Mengetahui nilai unik yang ada dalam kolom customer_status
unique_statuses = batch['customer_status'].unique()
print(unique_statuses)

# Validasi untuk memastikan value equipment_type
batch.expect_column_values_to_be_in_set(
    column='customer_status',
    value_set=['active', 'pending','inactive'],
    condition_parser='pandas',
    meta={"KPI": "Validation"}
) 

['active' 'pending' 'unknown' 'inactive']


{
  "exception_info": null,
  "meta": {
    "KPI": "Validation"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 6,
    "unexpected_percent": 30.0,
    "unexpected_percent_nonmissing": 30.0,
    "partial_unexpected_list": [
      "unknown",
      "unknown",
      "unknown",
      "unknown",
      "unknown",
      "unknown"
    ]
  }
}

In [397]:
# E2. KATEGORI VALID: checking equipment_type value yang valid

# Mengetahui nilai unik yang ada dalam kolom equipment_type
unique_statuses = batch['equipment_type'].unique()
print(unique_statuses)

# Validasi untuk memastikan value equipment_type
batch.expect_column_values_to_be_in_set(
    column='equipment_type',
    value_set=['Fiber Modem', '4G Router','5G Antenna'],
    condition_parser='pandas',
    meta={"KPI": "Validation"}
) 

['Fiber Modem' '4G Router' 'Unknown Device' '5G Antenna' 'ABC' nan]


{
  "exception_info": null,
  "meta": {
    "KPI": "Validation"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 2,
    "missing_percent": 10.0,
    "unexpected_count": 4,
    "unexpected_percent": 20.0,
    "unexpected_percent_nonmissing": 22.22222222222222,
    "partial_unexpected_list": [
      "Unknown Device",
      "ABC",
      "Unknown Device",
      "Unknown Device"
    ]
  }
}

In [398]:
# E3. DEPENDENSI ANTAR KOLOM: customer_status active maka location AND equipment_type not null
##################################### TIDAK BISA MASUK EXPECTATION #########################

# Filter baris dengan customer_status == 'active'
active_batch = batch[batch["customer_status"] == "active"]

# Validasi kolom location dan equipment_type tidak null hanya untuk customer_status == 'active'
active_batch["validation_condition"] = (
    active_batch["location"].notnull() &
    active_batch["equipment_type"].notnull()
)

# Validasi kondisi keseluruhan
active_batch.expect_column_values_to_be_in_set(
    column="validation_condition",
    value_set=[True],  # Hanya menerima nilai True sebagai valid
    meta={"KPI": "Validation"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Validation"
  },
  "success": false,
  "result": {
    "element_count": 3,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 2,
    "unexpected_percent": 66.66666666666666,
    "unexpected_percent_nonmissing": 66.66666666666666,
    "partial_unexpected_list": [
      false,
      false
    ]
  }
}

### F. **Uniqueness (Keunikan Data)**
Memastikan tidak ada duplikasi yang tidak perlu dalam dataset.

In [399]:
# F1. DUPLIKASI RECORD: tidak ada row sepenuhnya duplikat

duplicates_count = batch.duplicated().sum()
batch['duplicates_check'] = batch.duplicated()

# Validasi untuk memastikan tidak ada duplikasi antar baris
batch.expect_column_values_to_be_in_set(
    column="duplicates_check",  # Kolom yang sudah ditambahkan untuk pemeriksaan duplikasi
    value_set=[False],  # Tidak ada duplikasi yang boleh ada (harus False)
    meta={"KPI": "Uniqueness"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Uniqueness"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 1,
    "unexpected_percent": 5.0,
    "unexpected_percent_nonmissing": 5.0,
    "partial_unexpected_list": [
      true
    ]
  }
}

In [400]:
# F2. UNIK DALAM GRUP: setiap location, ID user harus unik

# Menambahkan Expectation: Nilai customer_connection_id harus unik dalam setiap lokasi
batch.expect_compound_columns_to_be_unique(
    column_list=["location", "customer_connection_id"],
    result_format="COMPLETE",  # Format hasil validasi
    meta={"KPI": "Uniqueness"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Uniqueness"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 4,
    "unexpected_percent": 20.0,
    "unexpected_percent_nonmissing": 20.0,
    "partial_unexpected_list": [
      {
        "location": "Kabupaten Sumbawa Barat",
        "customer_connection_id": "CC0002"
      },
      {
        "location": null,
        "customer_connection_id": "CC0016"
      },
      {
        "location": null,
        "customer_connection_id": "CC0016"
      },
      {
        "location": "Kabupaten Sumbawa Barat",
        "customer_connection_id": "CC0002"
      }
    ],
    "details": {
      "partial_unexpected_counts_error": "partial_unexpected_counts requested, but requires a hashable type"
    },
    "partial_unexpected_index_list": [
      1,
      15,
      16,
      18
    ],
    "partial_unexpected_counts": [],
    "unexpected_list": [
      {
      

In [401]:
# F3. KOMBINASI KOLOM UNIK: location, ID, AND phone_number must unik
########################## TIDAK BISA MASUK EXPECTATION #########################

# Menambahkan Expectation: Nilai customer_connection_id harus unik dalam setiap lokasi
batch.expect_compound_columns_to_be_unique(
    column_list=["location", "customer_connection_id", "phone_number"],
    result_format="COMPLETE",  # Format hasil validasi
    meta={"KPI": "Uniqueness"}
)

{
  "exception_info": null,
  "meta": {
    "KPI": "Uniqueness"
  },
  "success": false,
  "result": {
    "element_count": 20,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 2,
    "unexpected_percent": 10.0,
    "unexpected_percent_nonmissing": 10.0,
    "partial_unexpected_list": [
      {
        "location": null,
        "customer_connection_id": "CC0016",
        "phone_number": "+62 (022) 717-0307"
      },
      {
        "location": null,
        "customer_connection_id": "CC0016",
        "phone_number": "+62 (022) 717-0307"
      }
    ],
    "details": {
      "partial_unexpected_counts_error": "partial_unexpected_counts requested, but requires a hashable type"
    },
    "partial_unexpected_index_list": [
      15,
      16
    ],
    "partial_unexpected_counts": [],
    "unexpected_list": [
      {
        "location": null,
        "customer_connection_id": "CC0016",
        "phone_number": "+62 (022) 717-0307"
      },
      {
        "locati

## 4. Penyimpanan dan Eksekusi Validasi
Melakukan beberapa proses untuk menyimpan suite ekspektasi dan menjalankan validasi data berdasarkan ekspektasi yang telah diterapkan.

In [402]:
#Mengambil ekspektasi yang ada
expectation_suite = batch.get_expectation_suite()

#Menyimpan ekspektasi yang telah dimodifikasi atau diverifikasi
batch.save_expectation_suite(filepath="D:/GXpython12/great_expectations/expectations/expectation_suiteMixed3.json")

#Menjalankan validasi dengan suite ekspektasi yang telah disimpan
results = context.run_validation_operator('action_list_operator', assets_to_validate=[batch])

#Menampilkan hasil validasi
print(results)

{
  "evaluation_parameters": null,
  "run_id": {
    "run_name": "20250121T051921.953675Z",
    "run_time": "2025-01-21T05:19:21.953675+00:00"
  },
  "validation_operator_config": {
    "class_name": "ActionListValidationOperator",
    "module_name": "great_expectations.validation_operators",
    "name": "action_list_operator",
    "kwargs": {
      "action_list": [
        {
          "name": "store_validation_result",
          "action": {
            "class_name": "StoreValidationResultAction"
          }
        },
        {
          "name": "store_evaluation_params",
          "action": {
            "class_name": "StoreEvaluationParametersAction"
          }
        },
        {
          "name": "update_data_docs",
          "action": {
            "class_name": "UpdateDataDocsAction"
          }
        }
      ],
      "result_format": {
        "result_format": "SUMMARY",
        "partial_unexpected_count": 20
      }
    }
  },
  "success": false,
  "run_results": {
    "Va

## 5. Persentase Keberhasilan Validasi Data (by KPI)
Menemukan dan memproses file validasi terbaru yang disimpan dalam format JSON, kemudian menghitung persentase keberhasilan berdasarkan hasil validasi dari data tersebut. Proses ini termasuk membaca file, memetakan data ke Key Performance Indicators (KPI), serta menghitung statistik persentase keberhasilan per KPI.

### A. Fungsi Path File Validasi Terbaru
Menemukan file validasi terbaru dari direktori yang ditentukan.

In [403]:
import os
import json
from datetime import datetime

def find_latest_validation_file(base_path):
    """
    Finds the latest validation file in the specified base path.
    
    Args:
        base_path (str): The base directory where validation files are stored.

    Returns:
        str: The full path to the latest validation file, or None if no file is found.
    """
    latest_file = None
    latest_time = None

    # Traverse the base path to find all JSON files in subdirectories
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.endswith('.json'):
                full_path = os.path.join(root, file)

                # Extract the modification time
                file_time = os.path.getmtime(full_path)

                # Update the latest file if this one is newer
                if latest_time is None or file_time > latest_time:
                    latest_time = file_time
                    latest_file = full_path

    return latest_file

def load_validation_results(file_path):
    """
    Loads validation results from a JSON file.
    
    Args:
        file_path (str): Path to the validation results JSON file.

    Returns:
        dict: The parsed JSON data.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Base path for validation files
base_validation_path = r"D:\GXpython12\great_expectations\uncommitted\validations"

# Find the latest validation file
dynamic_path = find_latest_validation_file(base_validation_path)

if dynamic_path:
    print(f"Latest validation file: {dynamic_path}")

    # Load the validation results
    validation_data = load_validation_results(dynamic_path)

    # Access the results
    results = validation_data.get("results", {})


Latest validation file: D:\GXpython12\great_expectations\uncommitted\validations\mixed_check\20250121T051921.953675Z\20250121T051921.953675Z\ge_batch_id=45789b78-d7b7-11ef-9881-ec914fe37d50.json


### B. Memuat Data Hasil Validasi
Fungsi ini digunakan untuk memuat hasil validasi dari file JSON yang ditemukan sebelumnya. Data hasil validasi kemudian dapat diolah lebih lanjut untuk menghitung KPI dan persentase keberhasilannya.

In [404]:
import json
from collections import defaultdict

# Menentukan path dinamis dengan fungsi yang sudah ada
base_validation_path = r"D:\GXpython12\great_expectations\uncommitted\validations"
dynamic_path = find_latest_validation_file(base_validation_path)

# Pastikan path ditemukan, jika tidak, beri penanganan error
if dynamic_path:
    # Buka file JSON dengan path yang ditemukan
    with open(dynamic_path, "r") as file:
        data = json.load(file)

results = data["results"]


### C. Memetakan Jenis Validasi ke KPI
Fungsi `map_to_kpi` memetakan jenis validasi ke KPI tertentu berdasarkan kolom yang divalidasi. Hal ini memungkinkan pemetaan hasil validasi ke area-area spesifik seperti "Completeness", "Consistency", atau "Accuracy" dsb.

In [405]:
def map_to_kpi(expectation_type, column, kwargs):
    # Custom logic untuk mapping ke KPI
    if expectation_type == "expect_column_values_to_not_be_null":
        return "Completeness"
    elif expectation_type == "expect_column_values_to_match_regex":
        if column in ["phone_number", "email", "registration_timestamp", "received_timestamp"]:
            return "Consistency"
    elif expectation_type == "expect_column_values_to_be_in_set":
        if column == "customer_status":
            return "Validation"
        elif column == "equipment_type":
            return "Validation"
        elif column == "location":
            return "Accuracy"
        elif column == "equipment_status_capacity_check":
            return "Consistency"
        elif column == "timestamp_difference_valid":
            return "Timeliness"
    elif expectation_type == "expect_column_values_to_be_between":
        if column == "capacity":
            return "Accuracy"
    elif expectation_type == "expect_compound_columns_to_be_unique":
        return "Uniqueness"
    elif expectation_type == "expect_column_values_to_be_unique":
        return "Uniqueness"
    
    # Default case, gunakan mapping global jika ada
    return kwargs.get("meta", {}).get("KPI", "other")


### D. Menghitung Persentase Keberhasilan per KPI
Mengakumulasi persentase keberhasilan dari setiap KPI dan menghitung rata-rata persentase keberhasilan untuk setiap kategori KPI yang ada.

In [406]:
# Inisialisasi akumulasi
kpi_totals = defaultdict(lambda: {"success_percent_sum": 0, "total_expectations": 0})

# Proses Data untuk menghitung presentase keberhasilan
for result in results:
    expectation_type = result["expectation_config"]["expectation_type"]
    column = result["expectation_config"]["kwargs"].get("column", "")
    kwargs = result["expectation_config"]["kwargs"]

    # Panggil fungsi map_to_kpi untuk mendapatkan KPI
    kpi = map_to_kpi(expectation_type, column, kwargs)

    # Abaikan jika tidak ada KPI yang sesuai
    if kpi == "other":
        continue

    # Ambil unexpected_percent (default 100 jika tidak ditemukan)
    unexpected_percent = result.get("result", {}).get("unexpected_percent", 100)
    success_percent = 100 - unexpected_percent  # Keberhasilan = 100 - unexpected_percent

    # Update akumulasi untuk KPI tersebut
    kpi_totals[kpi]["success_percent_sum"] += success_percent
    kpi_totals[kpi]["total_expectations"] += 1

# Hitung rata-rata persentase keberhasilan per KPI
kpi_percentages = {}
for kpi, totals in kpi_totals.items():
    if totals["total_expectations"] > 0:
        kpi_percentages[kpi] = (
            totals["success_percent_sum"] / totals["total_expectations"]
        )
    else:
        kpi_percentages[kpi] = 0.0


### E. Hasil Output
Setelah perhitungan selesai, kode akan mencetak persentase keberhasilan untuk setiap KPI yang dihitung. Ini memberikan wawasan tentang keberhasilan validasi dalam berbagai area data.

In [407]:
#Cetak hasil
print("Persentase keberhasilan per KPI:")
for kpi, percentage in kpi_percentages.items():
    print(f"{kpi}: {percentage:.2f}%")


Persentase keberhasilan per KPI:
Completeness: 94.09%
Consistency: 87.00%
Validation: 75.00%
Accuracy: 95.00%
Timeliness: 90.00%
Uniqueness: 90.00%


## 6. Koneksi dan Penyimpanan Data KPI ke PostgreSQL
Proses koneksi ke database PostgreSQL, pembuatan tabel jika belum ada, serta penyimpanan data KPI yang telah dihitung ke dalam tabel yang sesuai

### A. Koneksi ke Database PostgreSQL
Melakukan koneksi ke database PostgreSQL menggunakan library psycopg2

In [408]:
import psycopg2

try:
    conn = psycopg2.connect(
        host="localhost",
        database="data_quality",
        user="postgres",
        password="satamasafika123"
    )
    cursor = conn.cursor()

    # Tambahkan kode untuk eksekusi query di sini jika perlu
    print("Koneksi berhasil!")

except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Koneksi berhasil!


### B. Create table (if not exist)
Setelah koneksi berhasil, kode ini membuat tabel kpi_quality jika tabel tersebut belum ada di dalam database. Struktur tabel mencakup berbagai kolom untuk menyimpan informasi terkait KPI, seperti completeness, uniqueness, consistency, dan lainnya.

In [410]:
# Membuat tabel jika belum ada
create_table_query = """
CREATE TABLE IF NOT EXISTS kpi_quality (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    dataset_name VARCHAR(100),
    completeness NUMERIC,
    uniqueness NUMERIC,
    consistency NUMERIC,
    accuracy NUMERIC,
    validation NUMERIC,
    timeliness NUMERIC
);
"""
cursor.execute(create_table_query)
conn.commit()


### C. Tentukan Nama Dataset dan Simpan Data
Menentukan nama dataset (misalnya "registration_check") dan kemudian menyimpan data KPI ke dalam tabel tersebut. Data yang disimpan mencakup nilai persentase dari KPI yang telah dihitung

In [412]:
# Menentukan nama dataset secara manual
dataset_name = "registration_check"

# Menyimpan data ke tabel
query = """
    INSERT INTO kpi_quality (dataset_name, completeness, uniqueness, consistency, accuracy, validation, timeliness)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
data = (
    dataset_name,
    kpi_percentages.get("Completeness", 0.0),
    kpi_percentages.get("Uniqueness", 0.0),
    kpi_percentages.get("Consistency", 0.0),
    kpi_percentages.get("Accuracy", 0.0),
    kpi_percentages.get("Validation", 0.0),
    kpi_percentages.get("Timeliness", 0.0)
)
cursor.execute(query, data)
conn.commit()

print("Data KPI berhasil disimpan ke PostgreSQL.")


Data KPI berhasil disimpan ke PostgreSQL.
